In [ ]:
# Taken from sagemaker template text2text-flan-xl

### 5. Advanced features: How to use various advanced parameters to control the generated text

***
This model also supports many advanced parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **num_return_sequences:** Number of output sequences returned. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **seed:** Fix the randomized state for reproducibility. If specified, it must be an integer.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments

***

In [14]:
# Input must be a json
payload = {
    "text_inputs": "Tell me the steps to make a pizza",
    "max_length": 50,
    "num_return_sequences": 3,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response


query_response = query_endpoint_with_json_payload(
    json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
)


def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions["generated_texts"]
    return generated_text


generated_texts = parse_response_multiple_texts(query_response)
print(generated_texts)

['Using a pizza stone, place the dough on the stone. Spread the pizza sauce on the dough. Top with cheese and toppings of your choice. Bake in the oven at 450 degrees for 15 minutes.', 'Spread pizza sauce on a pizza pan. Sprinkle with cheese. Bake at 450 degrees for 20 minutes.', 'Step 1: Make a pizza. Step 2: Spread pizza sauce on the pizza. Step 3: Top with cheese. Step 4: Bake in oven. Step 5: Enjoy!']


### 6. Advanced features: How to use prompts engineering to solve different tasks

Below we demonstrate solving 5 key tasks with Flan T5 model. The tasks are: **text summarization**, **common sense reasoning / question answering**, **sentence classification**, **translation**, **pronoun resolution**.


<font color='red'>Note </font>. **The notebook in the following sections are particularly designed for Flan T5 models (small, base, large, xl). There are other models like T5-one-line-summary which are designed for text summarization in particular. In that case, such models cannot perform all the following tasks.**

### 6.1. Summarization

Define the text article you want to summarize.

In [15]:
text = """Amazon Comprehend uses natural language processing (NLP) to extract insights about the content of documents. It develops insights by recognizing the entities, key phrases, language, sentiments, and other common elements in a document. Use Amazon Comprehend to create new products based on understanding the structure of documents. For example, using Amazon Comprehend you can search social networking feeds for mentions of products or scan an entire document repository for key phrases. 
You can access Amazon Comprehend document analysis capabilities using the Amazon Comprehend console or using the Amazon Comprehend APIs. You can run real-time analysis for small workloads or you can start asynchronous analysis jobs for large document sets. You can use the pre-trained models that Amazon Comprehend provides, or you can train your own custom models for classification and entity recognition. 
All of the Amazon Comprehend features accept UTF-8 text documents as the input. In addition, custom classification and custom entity recognition accept image files, PDF files, and Word files as input. 
Amazon Comprehend can examine and analyze documents in a variety of languages, depending on the specific feature. For more information, see Languages supported in Amazon Comprehend. Amazon Comprehend's Dominant language capability can examine documents and determine the dominant language for a far wider selection of languages."""

In [16]:
prompts = [
    "Briefly summarize this sentence: {text}",
    "Write a short summary for this text: {text}",
    "Generate a short summary this sentence:\n{text}",
    "{text}\n\nWrite a brief summary in a sentence or less",
    "{text}\nSummarize the aforementioned text in a single phrase.",
    "{text}\nCan you generate a short summary of the above paragraph?",
    "Write a sentence based on this summary: {text}",
    "Write a sentence based on '{text}'",
    "Summarize this article:\n\n{text}",
]

num_return_sequences = 3
parameters = {
    "max_length": 50,
    "num_return_sequences": num_return_sequences,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}

print(f"{bold}Number of return sequences are set as {num_return_sequences}{unbold}{newline}")
for each_prompt in prompts:
    payload = {"text_inputs": each_prompt.replace("{text}", text), **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} For prompt: '{each_prompt}'{unbold}{newline}")
    print(f"{bold} The {num_return_sequences} summarized results are{unbold}:{newline}")
    for idx, each_generated_text in enumerate(generated_texts):
        print(f"{bold}Result {idx}{unbold}: {each_generated_text}{newline}")

Number of return sequences are set as 3

 For prompt: 'Briefly summarize this sentence: {text}'

 The 3 summarized results are:

Result 0: Understand how Amazon Comprehend works. Learn how to use Amazon Comprehend.

Result 1: Learn about Amazon Comprehend. Use Amazon Comprehend to analyze documents. Use Amazon Comprehend to analyze documents in a variety of languages.

Result 2: Understand how Amazon Comprehend works. Use Amazon Comprehend to analyze documents. Use Amazon Comprehend to analyze documents in a variety of languages.

 For prompt: 'Write a short summary for this text: {text}'

 The 3 summarized results are:

Result 0: Amazon Comprehend uses natural language processing (NLP) to extract insights about the content of documents. Use Amazon Comprehend to create new products based on understanding the structure of documents. Access Amazon Comprehend document analysis

Result 1: Amazon Comprehend is a natural language processing (NLP) tool that extracts insights about the content

### 6.2. Common sense reasoning / natural language inference

In the common sense reasoning, you can design a prompt and combine it with the premise, hypothesis, and options, send the combined text into the endpoint to get an answer. Examples are demonstrated as below.

Define the premise, hypothesis, and options that you hope the model to reason.

In [17]:
premise = "The world cup has kicked off in Los Angeles, United States."
hypothesis = "The world cup takes place in United States."
options = """["yes", "no"]"""

In [18]:
prompts = [
    """{premise}\n\nBased on the paragraph above can we conclude that "\"{hypothesis}\"?\n\n{options_}""",
    """{premise}\n\nBased on that paragraph can we conclude that this sentence is true?\n{hypothesis}\n\n{options_}""",
    """{premise}\n\nCan we draw the following conclusion?\n{hypothesis}\n\n{options_}""",
    """{premise}\nDoes this next sentence follow, given the preceding text?\n{hypothesis}\n\n{options_}""",
    """{premise}\nCan we infer the following?\n{hypothesis}\n\n{options_}""",
    """Read the following paragraph and determine if the hypothesis is true:\n\n{premise}\n\nHypothesis: {hypothesis}\n\n{options_}""",
    """Read the text and determine if the sentence is true:\n\n{premise}\n\nSentence: {hypothesis}\n\n{options_}""",
    """Can we draw the following hypothesis from the context? \n\nContext:\n\n{premise}\n\nHypothesis: {hypothesis}\n\n{options_}""",
    """Determine if the sentence is true based on the text below:\n{hypothesis}\n\n{premise}\n{options_}""",
]

parameters = {
    "max_length": 50,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{premise}", premise)
    input_text = input_text.replace("{hypothesis}", hypothesis)
    input_text = input_text.replace("{options_}", options)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The reasoning result is{unbold}: '{generated_texts}'{newline}")

 For prompt: 'The world cup has kicked off in Los Angeles, United States.

Based on the paragraph above can we conclude that ""The world cup takes place in United States."?

["yes", "no"]'

 The reasoning result is: '['yes']'

 For prompt: 'The world cup has kicked off in Los Angeles, United States.

Based on that paragraph can we conclude that this sentence is true?
The world cup takes place in United States.

["yes", "no"]'

 The reasoning result is: '['yes']'

 For prompt: 'The world cup has kicked off in Los Angeles, United States.

Can we draw the following conclusion?
The world cup takes place in United States.

["yes", "no"]'

 The reasoning result is: '['no']'

 For prompt: 'The world cup has kicked off in Los Angeles, United States.
Does this next sentence follow, given the preceding text?
The world cup takes place in United States.

["yes", "no"]'

 The reasoning result is: '['yes']'

 For prompt: 'The world cup has kicked off in Los Angeles, United States.
Can we infer the f

### 6.3. Question and Answering

Now, let's try another reasoning task with a different type of prompt template. You can simply provide context and question as shown below.

In [ ]:
context = """The newest and most innovative Kindle yet lets you take notes on millions of books and documents, write lists and journals, and more. 

For readers who have always wished they could write in their eBooks, Amazon’s new Kindle lets them do just that. The Kindle Scribe is the first Kindle for reading and writing and allows users to supplement their books and documents with notes, lists, and more.

Here’s everything you need to know about the Kindle Scribe, including frequently asked questions.

The Kindle Scribe makes it easy to read and write like you would on paper 

The Kindle Scribe features a 10.2-inch, glare-free screen (the largest of all Kindle devices), crisp 300 ppi resolution, and 35 LED front lights that automatically adjust to your environment. Further personalize your experience with the adjustable warm light, font sizes, line spacing, and more.

It comes with your choice of the Basic Pen or the Premium Pen, which you use to write on the screen like you would on paper. They also attach magnetically to your Kindle and never need to be charged. The Premium Pen includes a dedicated eraser and a customizable shortcut button.

The Kindle Scribe has the most storage options of all Kindle devices: choose from 8 GB, 16 GB, or 32 GB to suit your level of reading and writing.
"""
question = "what are the key features of new Kindle?"

In [ ]:
prompts = [
    """Answer based on context:\n\n{context}\n\n{question}""",
    """{context}\n\nAnswer this question based on the article: {question}""",
    """{context}\n\n{question}""",
    """{context}\nAnswer this question: {question}""",
    """Read this article and answer this question {context}\n{question}""",
    """{context}\n\nBased on the above article, answer a question. {question}""",
    """Write an article that answers the following question: {question} {context}""",
]


parameters = {
    "max_length": 50,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{context}", context)
    input_text = input_text.replace("{question}", question)
    print(f"{bold} For prompt{unbold}: '{each_prompt}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The reasoning result is{unbold}: '{generated_texts}'{newline}")

### 6.4. Sentence / Sentiment Classification

Define the sentence you want to classify and the corresponded options.

In [ ]:
sentence = "This moive is so great and once again dazzles and delights us"
options_ = """OPTIONS:\n-positive \n-negative """

In [ ]:
prompts = [
    """Review:\n{sentence}\nIs this movie review sentence negative or positive?\n{options_}""",
    """Short movie review: {sentence}\nDid the critic think positively or negatively of the movie?\n{options_}""",
    """Sentence from a movie review: {sentence}\nWas the movie seen positively or negatively based on the preceding review? \n\n{options_}""",
    """\"{sentence}\"\nHow would the sentiment of this sentence be perceived?\n\n{options_}""",
    """Is the sentiment of the following sentence positive or negative?\n{sentence}\n{options_}""",
    """What is the sentiment of the following movie review sentence?\n{sentence}\n{options_}""",
]

parameters = {
    "max_length": 50,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sentence}", sentence)
    input_text = input_text.replace("{options_}", options_)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The reasoning result is{unbold}: '{generated_texts}'{newline}")

### 6.5. Translation

Define the sentence and the language you want to translate the sentence to.

In [ ]:
sent1 = "My name is Arthur"
lang2 = """German"""

In [ ]:
prompts = [
    """{sent1}\n\nTranslate to {lang2}""",
    """{sent1}\n\nCould you please translate this to {lang2}?""",
    """Translate to {lang2}:\n\n{sent1}""",
    """Translate the following sentence to {lang2}:\n{sent1}""",
    """How is \"{sent1}\" said in {lang2}?""",
    """Translate \"{sent1}\" to {lang2}?""",
]

parameters = {
    "max_length": 50,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sent1}", sent1)
    input_text = input_text.replace("{lang2}", lang2)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The translated result is{unbold}: '{generated_texts}'{newline}")

### 6.6. Pronoun resolution

Define the sentence, pronoun, and options you want to reason.

In [ ]:
sentence = "George talked to Mike because he had experiences in many aspects."
pronoun = "he"
options_ = """\n(A)George \n(B)Mike """

In [ ]:
prompts = [
    """sentence}\n\nWho is {pronoun} referring to?\n{options_}""",
    """{sentence}\n\nWho is \"{pronoun}\" in this prior sentence?\n{options_}""",
    """{sentence}\n\nWho is {pronoun} referring to in this sentence?\n{options_}""",
    """{sentence}\nTell me who {pronoun} is.\n{options_}""",
    """{sentence}\nBased on this sentence, who is {pronoun}?\n\n{options_}""",
    """Who is {pronoun} in the following sentence?\n\n{sentence}\n\n{options_}""",
    """Which entity is {pronoun} this sentence?\n\n{sentence}\n\n{options_}""",
]

parameters = {
    "max_length": 50,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{sentence}", sentence)
    input_text = input_text.replace("{pronoun}", pronoun)
    input_text = input_text.replace("{options_}", options_)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The reasoning result is{unbold}: '{generated_texts}'{newline}")

## 6.7. Imaginary article generation based on a title

In [ ]:
title = "University has new facility coming up"

In [ ]:
prompts = [
    """Title: \"{title}\"\\nGiven the above title of an imaginary article, imagine the article.\\n"""
]


parameters = {
    "max_length": 5000,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{title}", title)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The reasoning result is{unbold}: '{generated_texts}'{newline}")

## 6.8 Summarize a title based on the article

In [ ]:
article = """The newest and most innovative Kindle yet lets you take notes on millions of books and documents, write lists and journals, and more. 

For readers who have always wished they could write in their eBooks, Amazon’s new Kindle lets them do just that. The Kindle Scribe is the first Kindle for reading and writing and allows users to supplement their books and documents with notes, lists, and more.

Here’s everything you need to know about the Kindle Scribe, including frequently asked questions.

The Kindle Scribe makes it easy to read and write like you would on paper 

The Kindle Scribe features a 10.2-inch, glare-free screen (the largest of all Kindle devices), crisp 300 ppi resolution, and 35 LED front lights that automatically adjust to your environment. Further personalize your experience with the adjustable warm light, font sizes, line spacing, and more.

It comes with your choice of the Basic Pen or the Premium Pen, which you use to write on the screen like you would on paper. They also attach magnetically to your Kindle and never need to be charged. The Premium Pen includes a dedicated eraser and a customizable shortcut button.

The Kindle Scribe has the most storage options of all Kindle devices: choose from 8 GB, 16 GB, or 32 GB to suit your level of reading and writing."""

In [ ]:
prompts = ["""'\'{article} \n\n \\n\\nGive me a good title for the article above."""]

parameters = {
    "max_length": 2000,
    "num_return_sequences": 1,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
}


for each_prompt in prompts:
    input_text = each_prompt.replace("{article}", article)
    print(f"{bold} For prompt{unbold}: '{input_text}'{newline}")
    payload = {"text_inputs": input_text, **parameters}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)
    print(f"{bold} The reasoning result is{unbold}: '{generated_texts}'{newline}")